In [2]:
%load_ext cython

In [12]:
%%cython -a

# distutils: language = c++

from libc.stdlib cimport malloc, free
from libc.string cimport memset
from libcpp.vector cimport vector
from libcpp.algorithm cimport copy
from libcpp.pair cimport pair

from libcpp.unordered_map cimport unordered_map
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)

cdef void find_paths(int[:, :] graph_arcs, double[:,:] graph_data,  int num_arcs, int src, int dest,
                     int[:] visited, int[:] path, int path_index,
                     double[:] resources, double[:] resource_limits,
                     int num_resources, list all_paths):
    """
    Recursive function to find all paths from src to dest with resource constraints.
    """
    cdef int i, head, tail, r
    cdef bint valid
    visited[src] = 1
    path[path_index] = src
    path_index += 1

    if src == dest:
        all_paths.append([path[i] for i in range(path_index)])
    else:
        for i in range(num_arcs):
            head = graph_arcs[i, 0]
            tail = graph_arcs[i, 1]
            if head == src and not visited[tail]:
                valid = 1
                for r in range(num_resources):
                    resources[r] += graph_data[i, r]
                    if resources[r] > resource_limits[r]:
                        valid = 0

                if valid:
                    find_paths(graph_arcs, graph_data, num_arcs, tail, dest, visited, path, path_index, resources, resource_limits, num_resources, all_paths)

                # Backtrack resource consumption
                for r in range(num_resources):
                    resources[r] -= graph_data[i, r]

    # Backtrack
    visited[src] = 0
    path_index -= 1

cdef void propage(int direction, int parent_index, int parent_node,
                    vector[double]& r_max, 
                    vector[vector[int]]& adj_list,
                    double[:,:,:] graph_data,
                    vector[vector[vector[int]]]& L_p,
                    vector[vector[vector[double]]]& L_r, 
                    vector[vector[vector[int]]]& L_reach,
                    vector[vector[int]]& L_h,
                    vector[vector[int]]& L_ChildUp): 
    cdef int i, j, k, ind, child_node
    cdef int num_resources = r_max.size()+1
    
    cdef vector[int] child_path 
    cdef vector[double] child_resource 
    cdef vector[int] child_reachables 
    cdef int child_halfpoint

    for i in range(adj_list[parent_node].size()):
        if L_reach[parent_node][parent_index][adj_list[parent_node][i]] == 1:

            child_node = adj_list[parent_node][i]
            #build a new path
            child_path = L_p[parent_node][parent_index][:] 
            child_path.push_back(child_node)
            #build new resources and reachables
            child_resource = L_r[parent_node][parent_index][:]
            child_reachables = L_reach[parent_node][parent_index][:]
            child_reachables[child_node] = 0
            child_halfpoint = 0
            for j in range(num_resources):
                if direction == 1:
                    child_resource[j] += graph_data[parent_node, child_node, j]
                else:
                    child_resource[j] -= graph_data[child_node, parent_node, j]

            for  i in adj_list[child_node]:
                if child_reachables[i]==1:
                    for j in range(1,num_resources):
                        if (direction ==1 and child_resource[j]+graph_data[child_node,i,j] > r_max[j-1]) or (direction ==0 and child_resource[j]-graph_data[i,child_node,j] < 0):
                            child_reachables[i] = 0
                            break
            for j in range(1,num_resources):
                if (direction==1 and child_resource[j]>= r_max[j-1]/2) or (direction ==0  and child_resource[j]<= r_max[j-1]/2):
                    child_halfpoint = 1
                    break
                            
            #check if the child label is dominated by any of the existing labels
            k = 0
            while k < L_p[child_node].size():
                
                    ind = dominanceCheck(child_resource, L_r[child_node][k], child_reachables, L_reach[child_node][k])
                    if ind == 1:
                        break
                    elif ind == 2:
                        L_p[child_node].erase(L_p[child_node].begin()+k)
                        L_r[child_node].erase(L_r[child_node].begin()+k)
                        L_reach[child_node].erase(L_reach[child_node].begin()+k)
                        L_h[child_node].erase(L_h[child_node].begin()+k)
                    else:
                        k+=1
               

            if ind!=1:
                # TODO: need to 
                L_ChildUp[child_node].push_back(0)
                L_ChildUp[parent_node][parent_index] = 1
                L_p[child_node].push_back(child_path)
                L_r[child_node].push_back(child_resource)
                L_reach[child_node].push_back(child_reachables)
                L_h[child_node].push_back(child_halfpoint)
    L_ChildUp[parent_node][parent_index] = 1
    

        
cdef int dominanceCheck(vector[double]& l1_res, vector[double]& l2_res, vector[int]& l1_reach, vector[int]& l2_reach):
    cdef int i, ind
    ind = which_label_first(l1_res, l2_res)
    if ind == 1:
        for i in range(l1_res.size()):
            if l1_res[i] > l2_res[i]:
                return 0
        for i in range(l1_reach.size()):
            if l1_reach[i] > l2_reach[i]:
                return 0
        return 1
    else:
        for i in range(l1_res.size()):
            if l1_res[i] < l2_res[i]:
                return 0
        for i in range(l1_reach.size()):
            if l1_reach[i] < l2_reach[i]:
                return 0
        return 2
    
    

cdef int which_label_first(vector[double]& l1_res, vector[double]& l2_res):
    cdef int i
    for i in range(l1_res.size()):
        if l1_res[i] < l2_res[i]:
            return 1
        elif l1_res[i]==l2_res[i]:
            return which_label_first(l1_res[i+1:], l2_res[i+1:])
        else:
            return 2

       
cdef int ContinuouPropagate(vector[vector[int]]& L_ChildUp, vector[vector[int]]& L_h, int num_nodes):
    cdef int i, j
    for i in range(num_nodes):
        for j in range(L_ChildUp[i].size()):
            if L_h[i][j] + L_ChildUp[i][j]<=1:
                return 1
    return 0

def generate_all_paths(int[:,:] graph_arcs, double[:, :, :] graph_data, int num_nodes, int src, int dest,
                       double[:] resource_limits):
    """
    Generate all possible paths in a graph from source to destination considering resource constraints.
    Returns:
        List of all paths satisfying resource constraints.
    """
    cdef int num_arcs = graph_data.shape[0]
    cdef int num_resources = len(resource_limits)+1
    cdef int i, j, v
    
    cdef int mom2calc[3]
    mom2calc[:] = [1, 2, 3]
    cdef vector[double] r_max
    cdef vector[vector[vector[int]]] fw_p,bw_p # Labels'  paths
    cdef vector[vector[vector[double]]] fw_r,bw_r # Labels' resources consumed: each vector is [c, r1, r2, ..., rn]
    cdef vector[vector[vector[int]]] fw_reach,bw_reach # Labels'  reachables: each vector is [False, True, True, ..., True] 
    cdef vector[vector[int]] fw_h, bw_h # Labels' halfpoints: each element is a boolean: True if label reached half-point or otherwie False
    cdef vector[vector[int]] fw_childUp, bw_childUp # Labels' Childup Status: each element is a boolean: True if label has all child-up otherwie False

    cdef vector[vector[int]] Out_list, In_list # Outgoing and Incoming arcs for each node

    for i in range(num_nodes):
        Out_list.push_back(vector[int]())
        In_list.push_back(vector[int]())
        fw_p.push_back(vector[vector[int]]())
        bw_p.push_back(vector[vector[int]]())
        fw_r.push_back(vector[vector[double]]())
        bw_r.push_back(vector[vector[double]]())
        fw_reach.push_back(vector[vector[int]]())
        bw_reach.push_back(vector[vector[int]]())
        fw_h.push_back(vector[int]())
        bw_h.push_back(vector[int]())
        fw_childUp.push_back(vector[int]())
        bw_childUp.push_back(vector[int]())

    for i in range(num_arcs):
        Out_list[graph_arcs[i, 0]].push_back(graph_arcs[i, 1])
        In_list[graph_arcs[i, 1]].push_back(graph_arcs[i, 0])
    
    for i in range(num_resources-1):
        r_max.push_back(resource_limits[i])
    
    #Build the first forward and backward labels
    fw_r[src].push_back(vector[double](num_resources, 0))
    bw_r[dest].push_back(vector[double](num_resources, 0))
    for j in range(1,num_resources+1):
        bw_r[dest][0][j] = r_max[j-1]
    fw_reach[src].push_back(vector[int](num_nodes, 1))
    bw_reach[dest].push_back(vector[int](num_nodes, 1))
    fw_reach[src][0][src] = 0
    bw_reach[dest][0][dest] = 0
    fw_p[src].push_back(vector[int](1, src))
    bw_p[dest].push_back(vector[int](1, dest))
    fw_h[src].push_back(0)
    bw_h[dest].push_back(0)
    fw_childUp[src].push_back(0)
    bw_childUp[dest].push_back(0)

    propage(direction=1, parent_index = 0, parent_node = src, 
             r_max = r_max, adj_list = Out_list, graph_data = graph_data, L_p = fw_p, L_r = fw_r,
             L_reach = fw_reach, L_h = fw_h, L_ChildUp = fw_childUp)
    propage(direction = 0, parent_index = 0, parent_node = dest, r_max = r_max, adj_list = In_list,
         graph_data = graph_data, L_p = bw_p, L_r = bw_r, L_reach = bw_reach, L_h = bw_h, L_ChildUp = bw_childUp)
    
    # # Call the recursive function
    # find_paths(graph_arcs,graph_data, num_arcs, src, dest, visited, path, path_index, resources, resource_limits, num_resources, all_paths)
    for v in range(num_nodes):
        if v!=src and v!=dest:
            for i in range(fw_p[v].size()):
                if fw_childUp[v][i] == 0 and fw_h[v][i] == 0:
                    print("here forward")
                    propage(direction=1, parent_index = i, parent_node = v,r_max = r_max, adj_list = Out_list, graph_data = graph_data, L_p = fw_p, L_r = fw_r, L_reach = fw_reach, L_h = fw_h, L_ChildUp = fw_childUp)
                    break

                    
            for i in range(bw_p[v].size()):
                if bw_childUp[v][i] == 0 and bw_h[v][i] == 0:
                    print("here backward")
                    propage(direction = 0, parent_index = i, parent_node = v, r_max = r_max, adj_list = In_list,graph_data = graph_data, L_p = bw_p, L_r = bw_r, L_reach = bw_reach, L_h = bw_h, L_ChildUp = bw_childUp)
                    break
    
    return {i:[fw_p[i][j][k] for k in range(fw_p[i][j].size())] for i in range(fw_p.size()) for j in range(fw_p[i].size())}

Content of stdout:
_cython_magic_b9aa7ad2b483b3e021c0200a9d12024544a64b63.cpp
c:\Users\rezam\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\rezam\.ipython\cython\_cython_magic_b9aa7ad2b483b3e021c0200a9d12024544a64b63.cpp(20880): warning C4267: '=': conversion from 'size_t' to 'int', possible loss of data
C:\Users\rezam\.ipython\cython\_cython_magic_b9aa7ad2b483b3e021c0200a9d12024544a64b63.cpp(22319): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\rezam\.ipython\cython\_cython_magic_b9aa7ad2b483b3e021c0200a9d12024544a64b63.cpp(22329): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\rezam\.ipython\cython\_cython_magic_b9aa7ad2b483b3e021c0200a9d12024544a64b63.cpp(28649): warning C4551: function call missing argument list
   

In [ ]:
cdef int num_resources = r_max.size() + 1

cdef void build_new_path_resources_reachables(
    int parent_node, int parent_index, int child_node, int direction,
    double[:,:,:] graph_data, vector[vector[vector[int]]]& L_p,
    vector[vector[vector[double]]]& L_r, vector[vector[vector[int]]]& L_reach,
    vector[int]& child_path, vector[double]& child_resource, vector[int]& child_reachables,
    int& child_halfpoint, int num_resources):

    child_path = L_p[parent_node][parent_index][:]
    child_path.push_back(child_node)

    child_resource = L_r[parent_node][parent_index][:]
    child_reachables = L_reach[parent_node][parent_index][:]
    child_reachables[child_node] = 0
    child_halfpoint = 0

    for j in range(num_resources):
        if direction == 1:
            child_resource[j] += graph_data[parent_node, child_node, j]
        else:
            child_resource[j] -= graph_data[child_node, parent_node, j]

cdef void update_reachables(
    int child_node, int direction, double[:,:,:] graph_data,
    vector[int]& child_reachables, vector[double]& child_resource, int num_resources):

    for i in adj_list[child_node]:
        if child_reachables[i] == 1:
            for j in range(1, num_resources):
                if (direction == 1 and child_resource[j] + graph_data[child_node, i, j] > r_max[j-1]) or (direction == 0 and child_resource[j] - graph_data[i, child_node, j] < 0):
                    child_reachables[i] = 0
                    break

cdef int check_halfpoint(
    int direction, vector[double]& child_resource, int num_resources):

    for j in range(1, num_resources):
        if (direction == 1 and child_resource[j] >= r_max[j-1] / 2) or (direction == 0 and child_resource[j] <= r_max[j-1] / 2):
            return 1
    return 0

cdef void check_dominance(
    int child_node, vector[double]& child_resource, vector[int]& child_reachables,
    vector[vector[vector[int]]]& L_p, vector[vector[vector[double]]]& L_r,
    vector[vector[vector[int]]]& L_reach):

    k = 0
    while k < L_p[child_node].size():
        ind = dominanceCheck(child_resource, L_r[child_node][k], child_reachables, L_reach[child_node][k])
        if ind == 1:
            break
        elif ind == 2:
            L_p[child_node].erase(L_p[child_node].begin() + k)
            L_r[child_node].erase(L_r[child_node].begin() + k)
            L_reach[child_node].erase(L_reach[child_node].begin() + k)
        else:
            k += 1

cdef void process_node(
    int parent_node, int parent_index, int direction, double[:,:,:] graph_data,
    vector[vector[vector[int]]]& L_p, vector[vector[vector[double]]]& L_r,
    vector[vector[vector[int]]]& L_reach, vector[vector[int]]& adj_list):

    cdef int i, child_node
    cdef vector[int] child_path 
    cdef vector[double] child_resource 
    cdef vector[int] child_reachables 
    cdef int child_halfpoint

    for i in range(adj_list[parent_node].size()):
        if L_reach[parent_node][parent_index][adj_list[parent_node][i]] == 1:
            child_node = adj_list[parent_node][i]
            print(f"Processing child node: {child_node}")

            build_new_path_resources_reachables(
                parent_node, parent_index, child_node, direction, graph_data,
                L_p, L_r, L_reach, child_path, child_resource, child_reachables,
                child_halfpoint, num_resources)

            update_reachables(child_node, direction, graph_data, child_reachables, child_resource, num_resources)

            child_halfpoint = check_halfpoint(direction, child_resource, num_resources)

            check_dominance(child_node, child_resource, child_reachables, L_p, L_r, L_reach)

In [5]:
import numpy as np
import random

# Example graph data: [head, tail, weight, resource1, resource2]
n = 5
graph_arcs = np.array([[i,j] for i in range(n) for j in range(n) if i!=j], dtype=np.int32)

graph_data = np.random.randint(1,5, (n,n,3)).astype(np.float64)
for i in range(3):
    graph_data[:,:,i] += graph_data[:,:,i].T
# Define constraints
num_nodes = 11
src = 0
dest = 10
resource_limits = np.array([15.0, 13.0], dtype=np.float64)  # Constraints on resources 1 and 2

# Generate all paths
paths = generate_all_paths(graph_arcs, graph_data, num_nodes, src, dest, resource_limits)
print("Valid paths:", paths)


here forward
here forward
Valid paths: {0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 3], 4: [0, 4]}


: 

In [58]:
def find_paths_py(graph_data, num_nodes, src, dest, visited, path, path_index,
                  resources, resource_limits, num_resources, all_paths):
    """
    Recursive function to find all paths from src to dest with resource constraints in Python.
    """
    visited[src] = True
    path[path_index] = src
    path_index += 1

    if src == dest:
        all_paths.append(path[:path_index])
    else:
        for row in graph_data:
            head, tail = int(row[0]), int(row[1])
            if head == src and not visited[tail]:
                valid = True
                for r in range(num_resources):
                    resources[r] += row[2+ r]
                    if resources[r] > resource_limits[r]:
                        valid = False

                if valid:
                    find_paths_py(graph_data, num_nodes, tail, dest, visited, path, 
                                  path_index, resources, resource_limits, num_resources, all_paths)

                # Backtrack resource consumption
                for r in range(num_resources):
                    resources[r] -= row[2+ r]

    # Backtrack
    visited[src] = False
    path_index -= 1


def generate_all_paths_py(graph_data, num_nodes, src, dest, resource_limits):
    """
    Generate all possible paths in a graph from source to destination considering resource constraints.

    Arguments:
        graph_data: 2D list where each row represents [head, tail, weight, resource_1, resource_2, ...].
        num_nodes: Total number of nodes in the graph.
        src: Source node index.
        dest: Destination node index.
        resource_limits: List of resource constraints.

    Returns:
        List of all paths satisfying resource constraints.
    """
    visited = [False] * num_nodes
    path = [0] * num_nodes
    resources = [0.0] * len(resource_limits)
    all_paths = []
    path_index = 0
    num_resources = len(resource_limits)

    # Start recursion
    find_paths_py(graph_data, num_nodes, src, dest, visited, path, path_index,
                  resources, resource_limits, num_resources, all_paths)

    return all_paths


In [59]:
paths = generate_all_paths_py(graph_data_, num_nodes, src, dest, resource_limits)
print("Valid paths:", paths)

Valid paths: [[0, 1, 2, 3, 9], [0, 1, 2, 9], [0, 1, 3, 2, 9], [0, 1, 3, 4, 9], [0, 1, 3, 5, 9], [0, 1, 3, 6, 9], [0, 1, 3, 7, 6, 9], [0, 1, 3, 7, 9], [0, 1, 3, 8, 9], [0, 1, 3, 9], [0, 1, 4, 3, 9], [0, 1, 4, 5, 9], [0, 1, 4, 8, 9], [0, 1, 4, 9], [0, 1, 5, 2, 9], [0, 1, 5, 3, 9], [0, 1, 5, 4, 3, 9], [0, 1, 5, 4, 9], [0, 1, 5, 6, 9], [0, 1, 5, 7, 3, 9], [0, 1, 5, 7, 6, 9], [0, 1, 5, 7, 9], [0, 1, 5, 8, 3, 9], [0, 1, 5, 8, 9], [0, 1, 5, 9], [0, 1, 6, 9], [0, 1, 7, 6, 9], [0, 1, 8, 2, 9], [0, 1, 8, 3, 9], [0, 1, 8, 5, 9], [0, 1, 8, 9], [0, 1, 9], [0, 2, 1, 3, 9], [0, 2, 1, 5, 9], [0, 2, 1, 9], [0, 2, 3, 1, 9], [0, 2, 3, 4, 9], [0, 2, 3, 5, 9], [0, 2, 3, 7, 9], [0, 2, 3, 9], [0, 2, 5, 9], [0, 2, 6, 1, 9], [0, 2, 6, 9], [0, 2, 7, 1, 9], [0, 2, 7, 3, 9], [0, 2, 7, 5, 9], [0, 2, 7, 6, 1, 9], [0, 2, 7, 6, 4, 9], [0, 2, 7, 6, 9], [0, 2, 7, 9], [0, 2, 8, 9], [0, 2, 9], [0, 3, 1, 5, 9], [0, 3, 1, 8, 9], [0, 3, 1, 9], [0, 3, 2, 1, 5, 9], [0, 3, 2, 1, 9], [0, 3, 2, 6, 1, 9], [0, 3, 2, 6, 9], [0, 3, 

In [60]:
%timeit generate_all_paths(graph_arcs, graph_data, num_nodes, src, dest, resource_limits)


989 μs ± 85.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:

%timeit generate_all_paths_py(graph_data_, num_nodes, src, dest, resource_limits)

104 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%cython -a
# distutils: language = c++


from libc.stdlib cimport malloc, free
from libc.string cimport memset
from libcpp.vector cimport vector
from libcpp.algorithm cimport copy
from libcpp.pair cimport pair
from libcpp.unordered_map cimport unordered_map
import numpy as np
cimport numpy as np

def run(int num_resources, int num_nodes, int num_arcs, double[:] r_max,
             double[:,:,:] r, int[:, :] arcs):
    cdef int fw_p[1000000][100][100] 
    cdef int bw_p[1000000][100][100]

    cdef double fw_r[1000000][100][100]
    cdef double bw_r[1000000][100][100]

    cdef bint fw_h[1000000][1][100]
    cdef bint bw_h[1000000][1][100]
    
    cdef bint fw_reach[1000000][100][100]
    cdef bint bw_reach[1000000][100][100]
    
    cdef bint fw_open[1000000][100][100]
    cdef bint bw_open[1000000][100][100]

    cdef vector[vector[int]] OutList, InList
    cdef size_t i, j, k

    #TODO add edges to OutList and InList
    #TODO write propagate

cdef void Propagate(bint direction, size_t parent_index, int p_v,
                     double[:,:,:]& r, double[:]& r_max, vector[vector[int]] adj,
                     double[:,:,:]& L_r, int[:,:,:]& L_p, bint[:,:,:]& L_h, bint[:,:,:]& L_reach, bint[:,:,:]& L_open):
    cdef int i,j,k, v
    for i in range(adj[p_v].size()):
        v = adj[p_v][i]
        for j in range(1000000):
            if 



    






    # mom2calc[:] = [1, 2, 3]

Content of stdout:
_cython_magic_52be619ab96b32cb59926d470fe91faaa3d4ce89.cpp
c:\Users\rmirjali\AppData\Local\anaconda3\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\rmirjali\.ipython\cython\_cython_magic_52be619ab96b32cb59926d470fe91faaa3d4ce89.cpp(24814): warning C4551: function call missing argument list
   Creating library C:\Users\rmirjali\.ipython\cython\Users\rmirjali\.ipython\cython\_cython_magic_52be619ab96b32cb59926d470fe91faaa3d4ce89.cp312-win_amd64.lib and object C:\Users\rmirjali\.ipython\cython\Users\rmirjali\.ipython\cython\_cython_magic_52be619ab96b32cb59926d470fe91faaa3d4ce89.cp312-win_amd64.exp
Generating code
Finished generating code

In [5]:


obj = MyClass(10)
print(obj.get_value())  # Output: 10
obj.set_value(20)
print(obj.get_value())  # Output: 20

10
20
